# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Initialize-Environment" data-toc-modified-id="Initialize-Environment-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Initialize Environment</a></div><div class="lev1 toc-item"><a href="#Generate-list-of-data" data-toc-modified-id="Generate-list-of-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Generate list of data</a></div><div class="lev1 toc-item"><a href="#Task-Related-Constrasts" data-toc-modified-id="Task-Related-Constrasts-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Task-Related Constrasts</a></div><div class="lev2 toc-item"><a href="#Between-Task-Subgraph-Contrast" data-toc-modified-id="Between-Task-Subgraph-Contrast-31"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Between-Task Subgraph Contrast</a></div><div class="lev2 toc-item"><a href="#Within-Task-Subgraph-Contrast" data-toc-modified-id="Within-Task-Subgraph-Contrast-32"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Within-Task Subgraph Contrast</a></div><div class="lev1 toc-item"><a href="#Task-Performance-and-Subgraph-Expression" data-toc-modified-id="Task-Performance-and-Subgraph-Expression-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Task Performance and Subgraph Expression</a></div><div class="lev2 toc-item"><a href="#Load-Behavioral-Data" data-toc-modified-id="Load-Behavioral-Data-41"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Load Behavioral Data</a></div><div class="lev2 toc-item"><a href="#Reformulate-into-Data-Table" data-toc-modified-id="Reformulate-into-Data-Table-42"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Reformulate into Data Table</a></div><div class="lev2 toc-item"><a href="#Predicting-Interference-RT-(Hi)" data-toc-modified-id="Predicting-Interference-RT-(Hi)-43"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Predicting Interference RT (Hi)</a></div><div class="lev2 toc-item"><a href="#Predicting-Cost-RT-(Hi-Lo)" data-toc-modified-id="Predicting-Cost-RT-(Hi-Lo)-44"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Predicting Cost RT (Hi-Lo)</a></div><div class="lev2 toc-item"><a href="#Predicting-Interference-Accuracy-(Hi)" data-toc-modified-id="Predicting-Interference-Accuracy-(Hi)-45"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Predicting Interference Accuracy (Hi)</a></div><div class="lev2 toc-item"><a href="#Predicting-Cost-Accuracy-(Hi-Lo)" data-toc-modified-id="Predicting-Cost-Accuracy-(Hi-Lo)-46"><span class="toc-item-num">4.6&nbsp;&nbsp;</span>Predicting Cost Accuracy (Hi-Lo)</a></div>

# Initialize Environment

In [ ]:
try:
    %load_ext autoreload
    %autoreload 2
    %reset
except:
    print 'NOT IPYTHON'

from __future__ import division

import os
os.environ['MKL_NUM_THREADS'] = '1'
os.environ['NUMEXPR_NUM_THREADS'] = '1'
os.environ['OMP_NUM_THREADS'] = '1'
import sys
import glob

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import scipy.io as io
import h5py
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rcParams

sys.path.append('/Users/akhambhati/Developer/hoth_research/Echobase')
import Echobase
convert_conn_vec_to_adj_matr = Echobase.Network.Transforms.configuration.convert_conn_vec_to_adj_matr
convert_adj_matr_to_cfg_matr = Echobase.Network.Transforms.configuration.convert_adj_matr_to_cfg_matr

rcParams = Echobase.Plotting.fig_format.update_rcparams(rcParams)

path_Remotes = '/Users/akhambhati/Remotes'
path_CoreData = path_Remotes + '/CORE.fMRI_cogcontrol.medaglia'
path_PeriphData = path_Remotes + '/RSRCH.NMF_CogControl'
path_InpData = path_PeriphData + '/e02b-FuncSubg'
path_ExpData = path_PeriphData + '/e03-FuncSubg_Dynamics'

for path in [path_CoreData, path_PeriphData, path_ExpData]:
    if not os.path.exists(path):
        print('Path: {}, does not exist'.format(path))
        os.makedirs(path)

# Generate list of data

In [ ]:
%matplotlib inline

df_system = np.load('{}/Subgraph.11System.Filtered.npz'.format(path_InpData))
sys_subgraph = df_system['system_subgraph']
n_subgraph = len(sys_subgraph)
key_type = df_system['task_key']

# Task-Related Constrasts

## Between-Task Subgraph Contrast

In [ ]:
%matplotlib inline

grp_order = ['pos', 'neg']
tsk_order = ['Stroop', 'Navon']

# Run Statistics on Key and Group Level from Above
# Accumulate in fac_stat
fac_stat = []
for fac_ii in xrange(n_subgraph):
    grp_stat = []
                 
    for grp in grp_order:
        subplot_dict = {'title': grp,
                        'xlabel': [],
                        'distrib': [],
                        'stats': []}
        
        for tsk in tsk_order:
            key_rs_ix = np.flatnonzero(key_type == 'adj_rs_{}_{}'.format(grp, tsk))
            key_hi_ix = np.flatnonzero(key_type == 'adj_hi_{}_{}'.format(grp, tsk))
            key_lo_ix = np.flatnonzero(key_type == 'adj_lo_{}_{}'.format(grp, tsk))
            
            subj_rs = sys_subgraph[fac_ii]['expr_coef'][:, key_rs_ix, :].squeeze()
            subj_hi = sys_subgraph[fac_ii]['expr_coef'][:, key_hi_ix, :].squeeze()
            subj_lo = sys_subgraph[fac_ii]['expr_coef'][:, key_lo_ix, :].squeeze()

            mean_subj_coef = 1./2*(np.mean(subj_hi, axis=1) + np.mean(subj_lo, axis=1))
            
            subplot_dict['xlabel'].append(tsk)
            subplot_dict['distrib'].append(mean_subj_coef)
        
        for key_pair in [[0,1]]:
            ts, pv = stats.ttest_rel(subplot_dict['distrib'][key_pair[0]],
                                     subplot_dict['distrib'][key_pair[1]]) 
                            
            subplot_dict['stats'].append({'name': '{}-{}'.format(subplot_dict['xlabel'][key_pair[0]],
                                                                 subplot_dict['xlabel'][key_pair[1]]),
                                          'tstat': ts,
                                          'pval': pv,
                                          'sig': ''})
        grp_stat.append(subplot_dict)
    fac_stat.append(grp_stat)

# Benjamini-Hochberg Multiple Comparisons
all_pval = np.sort([kstat['pval']
                    for fstat in fac_stat
                    for gstat in fstat
                    for kstat in gstat['stats']])
alpha = 0.05
n_comp = len(all_pval)
for ix, i in enumerate(range(1, n_comp+1, 1)):
    adj_alpha = alpha/(n_comp+1-i)
    if all_pval[ix] > adj_alpha:
        benj_hoch_ix = ix
        break
print('Adjusted Alpha: %0.10f' % adj_alpha)

# Re-adjust significant indicator and re-order subgraphs
min_fac_pval = []
sel_fac_between_sig = []
for fac_ii, fstat in enumerate(fac_stat):
    pval_list = []
    for gstat in fstat:
        for kstat in gstat['stats']:
            if kstat['pval'] in all_pval[:benj_hoch_ix]:
                kstat['sig'] = '*'
                sel_fac_between_sig.append(fac_ii)
            pval_list.append(kstat['pval'])
    min_fac_pval.append(np.min(pval_list))
sel_fac_between_sig = np.unique(sel_fac_between_sig)

# Plot each result
plt.close()
for fac_ix in xrange(n_subgraph): 
    # Handle Subgraph Plotting
    print('****************************** Subgraph {} ******************************'.format(fac_ix+1))    
    
    # Handle Distribution Plotting
    fstat = fac_stat[fac_ix]
    for subp_ii, gstat in enumerate(fstat):
        ax = plt.subplot(2, 2, subp_ii+1)
        ax.boxplot(gstat['distrib'])
        ax.set_xticklabels(gstat['xlabel'])
        ax.set_ylabel('Expression')
        ax.set_title(gstat['title'])
        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')

        for kstat in gstat['stats']:
            print('    %s-%s: ts: %0.6f pval: %0.6f sig: %s' % 
                  (gstat['title'], kstat['name'],
                   kstat['tstat'], kstat['pval'],
                   kstat['sig']))        
    plt.savefig('./e03-Figures/Between_Task.Subgraph.{}.svg'.format(fac_ix+1))        
    plt.close()
    print('\n\n\n')

## Within-Task Subgraph Contrast

In [ ]:
%matplotlib inline

grp_order = ['pos_Stroop', 'pos_Navon', 'neg_Stroop', 'neg_Navon']
tsk_order = ['rs', 'lo', 'hi']

# Run Statistics on Key and Group Level from Above
# Accumulate in fac_stat
fac_stat = []
for fac_ii in xrange(n_subgraph):
    grp_stat = []
                 
    for grp in grp_order:
        subplot_dict = {'title': grp,
                        'xlabel': [],
                        'distrib': [],
                        'stats': []}
        
        for tsk in tsk_order:
            key_ix = np.flatnonzero(key_type == 'adj_{}_{}'.format(tsk, grp))
                        
            subj_coef = sys_subgraph[fac_ii]['expr_coef'][:, key_ix, :].squeeze()

            mean_subj_coef = np.mean(subj_coef, axis=1)
            
            subplot_dict['xlabel'].append(tsk)
            subplot_dict['distrib'].append(mean_subj_coef)
            
        for key_pair in [[1,0], [2,0], [2,1]]:
            ts, pv = stats.ttest_rel(subplot_dict['distrib'][key_pair[0]],
                                     subplot_dict['distrib'][key_pair[1]])    
            if np.isnan(pv):
                pv = 1.0

            subplot_dict['stats'].append({'name': '{}-{}'.format(subplot_dict['xlabel'][key_pair[0]],
                                                                 subplot_dict['xlabel'][key_pair[1]]),
                                          'tstat': ts,
                                          'pval': pv,
                                          'sig': ''})        
        
        grp_stat.append(subplot_dict)
    fac_stat.append(grp_stat)

# Benjamini-Hochberg Multiple Comparisons
all_pval = np.sort([kstat['pval']
                    for fstat in fac_stat
                    for gstat in fstat
                    for kstat in gstat['stats']])
alpha = 0.05
n_comp = len(all_pval)
for ix, i in enumerate(range(1, n_comp+1, 1)):
    adj_alpha = alpha/(n_comp+1-i)
    if all_pval[ix] > adj_alpha:
        benj_hoch_ix = ix
        break
print('Adjusted Alpha: %0.10f' % adj_alpha)

# Re-adjust significant indicator and re-order subgraphs
min_fac_pval = []
sel_fac_within_sig = []
for fac_ii, fstat in enumerate(fac_stat):
    pval_list = []
    for gstat in fstat:
        for kstat in gstat['stats']:
            if kstat['pval'] in all_pval[:benj_hoch_ix]:
                kstat['sig'] = '*'
                sel_fac_within_sig.append(fac_ii)
            pval_list.append(kstat['pval'])
    min_fac_pval.append(np.min(pval_list))
sel_fac_within_sig = np.unique(sel_fac_within_sig)

# Plot each result
plt.close()
for fac_ix in xrange(n_subgraph): 
    # Handle Subgraph Plotting
    print('****************************** Subgraph {} ******************************'.format(fac_ix+1))    

    # Handle Distribution Plotting
    fstat = fac_stat[fac_ix]
    for subp_ii, gstat in enumerate(fstat):
        ax = plt.subplot(2, 2, subp_ii+1)
        ax.boxplot(gstat['distrib'])
        ax.set_xticklabels(gstat['xlabel'])
        ax.set_ylabel('Expression')
        ax.set_title(gstat['title'])
        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')


        for kstat in gstat['stats']:
            print('    %s-%s: ts: %0.6f pval: %0.6f sig: %s' % 
                  (gstat['title'], kstat['name'],
                   kstat['tstat'], kstat['pval'],
                   kstat['sig']))
        print('')
    plt.savefig('./e03-Figures/Within_Task.Subgraph.{}.svg'.format(fac_ix+1))        
    plt.close()
    print('\n\n\n')

# Task Performance and Subgraph Expression 

## Load Behavioral Data

In [ ]:
df_beh = io.loadmat('{}/BehavioralData_CogControlCorrect.mat'.format(path_CoreData))
df_blk = io.loadmat('{}/BlockwiseDataCorrectTrialsOnly.mat'.format(path_CoreData))

stroop_lbl = [lbl[0] for lbl in df_beh['StroopLabels'][0]]
navon_lbl = [lbl[0] for lbl in df_beh['NavonLabels'][0]]
stroop_beh = df_beh['StroopCorrectOnly']
navon_beh = df_beh['NavonCorrectOnly']

# Removed subjects
bad_subj_ix = [1, 6]
good_subj_ix = np.setdiff1d(np.arange(stroop_beh.shape[0]), bad_subj_ix)

# Excessive movement
stroop_move = io.loadmat('{}/StroopMove.mat'.format(path_CoreData))['move'][:, 0]
navon_move = io.loadmat('{}/NavonMove.mat'.format(path_CoreData))['move'][:, 0]

# Reaction Time
stroop_interf_rt = stroop_beh[good_subj_ix, 6]
navon_interf_rt = navon_beh[good_subj_ix, 6]

stroop_noninterf_rt = stroop_beh[good_subj_ix, 8]
navon_noninterf_rt = navon_beh[good_subj_ix, 8]

stroop_cost_rt = stroop_interf_rt - stroop_noninterf_rt
navon_cost_rt = navon_interf_rt - navon_noninterf_rt

# Accuracy
stroop_interf_acc = stroop_beh[good_subj_ix, 1]
navon_interf_acc = navon_beh[good_subj_ix, 1]

stroop_noninterf_acc = stroop_beh[good_subj_ix, 2]
navon_noninterf_acc = navon_beh[good_subj_ix, 2]

stroop_cost_acc = stroop_noninterf_acc - stroop_interf_acc
navon_cost_acc = navon_noninterf_acc - navon_interf_acc

print('Stroop Labels: {}'.format(stroop_lbl))
print('Navon Labels: {}'.format(navon_lbl))

"""
'high control accuracy', 'low control accuracy', 'high control mean RT',
'high control median RT', 'low control mean RT', 'low control median RT'
"""

## Reformulate into Data Table

## Predicting Interference RT (Hi)

In [ ]:
%matplotlib inline

grp_order = ['Stroop', 'Navon']
tsk_order = ['hi_pos', 'hi_neg']        
beh_grp = {'Stroop': [stroop_move, stroop_interf_rt],
           'Navon': [navon_move, navon_interf_rt]}

# Run Statistics on Key and Group Level from Above
# Accumulate in fac_stat
fac_stat = []
for fac_ii in xrange(n_subgraph):
    grp_stat = []
                 
    for grp in grp_order:
        for tsk in tsk_order:
            key_ix = np.flatnonzero(key_type == 'adj_{}_{}'.format(tsk, grp))
            
            subj_coef = sys_subgraph[fac_ii]['expr_coef'][:, key_ix, :].squeeze()
            mean_subj_coef = np.mean(subj_coef, axis=-1)
            
            m, b, _, _, _ = stats.linregress(beh_grp[grp][0], mean_subj_coef)
            real_mean_subj_coef_nomove = mean_subj_coef - beh_grp[grp][0]*m+b
            
            m, b, _, _, _ = stats.linregress(beh_grp[grp][0], beh_grp[grp][1])
            beh_score_nomove = beh_grp[grp][1] - beh_grp[grp][0]*m+b            
            
            real_m, real_b, real_rho, real_pval, _ = stats.linregress(real_mean_subj_coef_nomove, beh_score_nomove)

            null_rho_dist = []
            for i in xrange(10000):
                null_rho, _ = stats.pearsonr(np.random.permutation(real_mean_subj_coef_nomove),
                                             beh_score_nomove)
                null_rho_dist.append(null_rho)
            null_rho_dist = np.array(null_rho_dist)
            if real_rho > 0:
                real_pval = np.mean(null_rho_dist > real_rho)
            else:
                real_pval = np.mean(null_rho_dist < real_rho)

            subplot_dict = {'title': '{}: {}'.format(grp, tsk),
                            'xlabel': tsk,
                            'distrib': (real_mean_subj_coef_nomove, beh_score_nomove),
                            'stats': []}
            subplot_dict['stats'].append({'rho': real_rho,
                                          'pval': real_pval,
                                          'slope': real_m,
                                          "interc": real_b,
                                          'sig': ''})
            
            grp_stat.append(subplot_dict)
    fac_stat.append(grp_stat)


# Benjamini-Hochberg Multiple Comparisons
all_pval = np.sort([kstat['pval']
                    for fstat in fac_stat
                    for gstat in fstat
                    for kstat in gstat['stats']])
alpha = 0.05
n_comp = len(all_pval)
for ix, i in enumerate(range(1, n_comp+1, 1)):
    adj_alpha = alpha/(n_comp+1-i)
    if all_pval[ix] > adj_alpha:
        benj_hoch_ix = ix
        break
print('Adjusted Alpha: %0.10f' % adj_alpha)

# Re-adjust significant indicator and re-order subgraphs
min_fac_pval = []
sel_fac_between_sig = []
for fac_ii, fstat in enumerate(fac_stat):
    pval_list = []
    for gstat in fstat:
        for kstat in gstat['stats']:
            #if kstat['pval'] in all_pval[:benj_hoch_ix]:
            if kstat['pval'] < alpha:
                kstat['sig'] = '*'
            pval_list.append(kstat['pval'])
    min_fac_pval.append(np.min(pval_list))
sel_fac_ord = np.argsort(min_fac_pval)

# Plot each result
plt.close()
for fac_ix in xrange(n_subgraph):
    # Handle Subgraph Plotting
    print('****************************** Subgraph {} ******************************'.format(fac_ix+1))    

    # Handle Distribution Plotting
    fstat = fac_stat[fac_ix]
    for subp_ii, gstat in enumerate(fstat):
        ax = plt.subplot(2, 2, subp_ii+1)
        ax.plot(gstat['distrib'][0], 
                gstat['stats'][0]['slope']*gstat['distrib'][0]+gstat['stats'][0]['interc'],
                color='k', alpha=0.75)
        ax.scatter(gstat['distrib'][0], gstat['distrib'][1],
                   color=[0.3, 0.3, 0.3])

        ax.set_xlabel('Mean Expression')
        ax.set_ylabel('High Demand Reaction Time')
        ax.set_title(gstat['title'])
        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')

        for kstat in gstat['stats']:
            print('    %12s: rho: %3.5f pval: %3.5f sig: %s' % 
              (gstat['title'],
               kstat['rho'],
               kstat['pval'],
               kstat['sig']))
        print('')
    plt.savefig('./e03-Figures/High_Demand_RT.Subgraph.{}.svg'.format(fac_ix+1))        
    plt.close()
    print('\n\n\n')            

## Predicting Cost RT (Hi-Lo)

In [ ]:
%matplotlib inline

grp_order = ['Stroop', 'Navon']
tsk_order = ['pos', 'neg']
beh_grp = {'Stroop': [stroop_move, stroop_cost_rt],
           'Navon': [navon_move, navon_cost_rt]}

# Run Statistics on Key and Group Level from Above
# Accumulate in fac_stat
fac_stat = []
for fac_ii in xrange(n_subgraph):
    grp_stat = []
                 
    for grp in grp_order:
        for tsk in tsk_order:
            key_hi_ix = np.flatnonzero(key_type == 'adj_hi_{}_{}'.format(tsk, grp))
            key_lo_ix = np.flatnonzero(key_type == 'adj_lo_{}_{}'.format(tsk, grp))
            
            subj_coef_hi = sys_subgraph[fac_ii]['expr_coef'][:, key_hi_ix, :].squeeze()
            subj_coef_lo = sys_subgraph[fac_ii]['expr_coef'][:, key_lo_ix, :].squeeze()
            mean_subj_coef = np.mean(subj_coef_hi, axis=1) - np.mean(subj_coef_lo, axis=1)
            
            m, b, _, _, _ = stats.linregress(beh_grp[grp][0], mean_subj_coef)
            real_mean_subj_coef_nomove = mean_subj_coef - beh_grp[grp][0]*m+b
            
            m, b, _, _, _ = stats.linregress(beh_grp[grp][0], beh_grp[grp][1])
            beh_score_nomove = beh_grp[grp][1] - beh_grp[grp][0]*m+b            
            
            real_m, real_b, real_rho, real_pval, _ = stats.linregress(real_mean_subj_coef_nomove, beh_score_nomove)
            
            null_rho_dist = []
            for i in xrange(10000):
                null_rho, _ = stats.pearsonr(np.random.permutation(real_mean_subj_coef_nomove),
                                             beh_score_nomove)
                null_rho_dist.append(null_rho)
            null_rho_dist = np.array(null_rho_dist)
            if real_rho > 0:
                real_pval = np.mean(null_rho_dist > real_rho)
            else:
                real_pval = np.mean(null_rho_dist < real_rho)
                
            subplot_dict = {'title': '{}: {}'.format(grp, tsk),
                            'xlabel': tsk,
                            'distrib': (real_mean_subj_coef_nomove, beh_score_nomove),
                            'stats': []}
            subplot_dict['stats'].append({'rho': real_rho,
                                          'pval': real_pval,
                                          'slope': real_m,
                                          "interc": real_b,
                                          'sig': ''})       
            
            grp_stat.append(subplot_dict)
    fac_stat.append(grp_stat)

# Benjamini-Hochberg Multiple Comparisons
all_pval = np.sort([kstat['pval']
                    for fstat in fac_stat
                    for gstat in fstat
                    for kstat in gstat['stats']])
alpha = 0.05
n_comp = len(all_pval)
for ix, i in enumerate(range(1, n_comp+1, 1)):
    adj_alpha = alpha/(n_comp+1-i)
    if all_pval[ix] > adj_alpha:
        benj_hoch_ix = ix
        break
print('Adjusted Alpha: %0.10f' % adj_alpha)

# Re-adjust significant indicator and re-order subgraphs
min_fac_pval = []
sel_fac_between_sig = []
for fac_ii, fstat in enumerate(fac_stat):
    pval_list = []
    for gstat in fstat:
        for kstat in gstat['stats']:
            #if kstat['pval'] in all_pval[:benj_hoch_ix]:
            if kstat['pval'] < alpha:
                kstat['sig'] = '*'
            pval_list.append(kstat['pval'])
    min_fac_pval.append(np.min(pval_list))
sel_fac_ord = np.argsort(min_fac_pval)

# Plot each result
plt.close()
for fac_ix in xrange(n_subgraph):
    # Handle Subgraph Plotting
    print('****************************** Subgraph {} ******************************'.format(fac_ix+1))    

    # Handle Distribution Plotting
    fstat = fac_stat[fac_ix]
    for subp_ii, gstat in enumerate(fstat):
        ax = plt.subplot(2, 2, subp_ii+1)
        ax.plot(gstat['distrib'][0], 
                gstat['stats'][0]['slope']*gstat['distrib'][0]+gstat['stats'][0]['interc'],
                color='k', alpha=0.75)
        ax.scatter(gstat['distrib'][0], gstat['distrib'][1],
                   color=[0.3, 0.3, 0.3])

        ax.set_xlabel('Mean Expression Change')
        ax.set_ylabel('Reaction Time Cost')
        ax.set_title(gstat['title'])
        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')

        for kstat in gstat['stats']:
            print('    %12s: rho: %3.5f pval: %3.5f sig: %s' % 
              (gstat['title'],
               kstat['rho'],
               kstat['pval'],
               kstat['sig']))
        print('')
    plt.savefig('./e03-Figures/Cost_RT.Subgraph.{}.svg'.format(fac_ix+1))        
    plt.close()
    print('\n\n\n')            

## Predicting Interference Accuracy (Hi)

In [ ]:
%matplotlib inline

grp_order = ['Stroop', 'Navon']
tsk_order = ['hi_pos', 'hi_neg']        
beh_grp = {'Stroop': [stroop_move, stroop_interf_acc],
           'Navon': [navon_move, navon_interf_acc]}

# Run Statistics on Key and Group Level from Above
# Accumulate in fac_stat
fac_stat = []
for fac_ii in xrange(n_subgraph):
    grp_stat = []
                 
    for grp in grp_order:
        for tsk in tsk_order:
            key_ix = np.flatnonzero(key_type == 'adj_{}_{}'.format(tsk, grp))
            
            subj_coef = fac_coef_subj[fac_ii, :, key_ix, :][0, ...]
            mean_subj_coef = np.mean(subj_coef, axis=-1)
            
            m, b, _, _, _ = stats.linregress(beh_grp[grp][0], mean_subj_coef)
            real_mean_subj_coef_nomove = mean_subj_coef - beh_grp[grp][0]*m+b
            
            m, b, _, _, _ = stats.linregress(beh_grp[grp][0], beh_grp[grp][1])
            beh_score_nomove = beh_grp[grp][1] - beh_grp[grp][0]*m+b            
            
            real_m, real_b, real_rho, real_pval, _ = stats.linregress(real_mean_subj_coef_nomove, beh_score_nomove)

            null_rho_dist = []
            for i in xrange(10000):
                null_rho, _ = stats.pearsonr(np.random.permutation(real_mean_subj_coef_nomove),
                                             beh_score_nomove)
                null_rho_dist.append(null_rho)
            null_rho_dist = np.array(null_rho_dist)
            if real_rho > 0:
                real_pval = np.mean(null_rho_dist > real_rho)
            else:
                real_pval = np.mean(null_rho_dist < real_rho)

            subplot_dict = {'title': '{}: {}'.format(grp, tsk),
                            'xlabel': tsk,
                            'distrib': (real_mean_subj_coef_nomove, beh_score_nomove),
                            'stats': []}
            subplot_dict['stats'].append({'rho': real_rho,
                                          'pval': real_pval,
                                          'slope': real_m,
                                          "interc": real_b,
                                          'sig': ''})
            
            grp_stat.append(subplot_dict)
    fac_stat.append(grp_stat)


# Benjamini-Hochberg Multiple Comparisons
all_pval = np.sort([kstat['pval']
                    for fstat in fac_stat
                    for gstat in fstat
                    for kstat in gstat['stats']])
alpha = 0.05
n_comp = len(all_pval)
for ix, i in enumerate(range(1, n_comp+1, 1)):
    adj_alpha = alpha/(n_comp+1-i)
    if all_pval[ix] > adj_alpha:
        benj_hoch_ix = ix
        break
print('Adjusted Alpha: %0.10f' % adj_alpha)

# Re-adjust significant indicator and re-order subgraphs
min_fac_pval = []
sel_fac_between_sig = []
for fac_ii, fstat in enumerate(fac_stat):
    pval_list = []
    for gstat in fstat:
        for kstat in gstat['stats']:
            if kstat['pval'] in all_pval[:benj_hoch_ix]:
                kstat['sig'] = '*'
            pval_list.append(kstat['pval'])
    min_fac_pval.append(np.min(pval_list))
sel_fac_ord = np.argsort(min_fac_pval)

# Plot each result
for fac_ix in sel_fac_ord:
    
    # Handle Subgraph Plotting
    f_sys_subgraph = sys_subgraph[fac_ix]
    #assert f_sys_subgraph['Subgraph_ID'] == (fac_ix+1)
    
    plt.figure(figsize=(12,12))
    ax = plt.subplot(3, 3, 1)
    ax.matshow(f_sys_subgraph['filt_system_subg'], cmap='rainbow')
    if len(df_subg['system_names']) < 20:
        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')

        ax.set_yticks(np.arange(len(df_subg['system_names'])))
        ax.set_xticks(np.arange(len(df_subg['system_names'])))
        ax.set_yticklabels(df_subg['system_names'])
        ax.set_xticklabels(df_subg['system_names'])
        _ = [tick.label.set_fontsize(12) for tick in ax.yaxis.get_major_ticks()]
        _ = [tick.label.set_fontsize(12) for tick in ax.xaxis.get_major_ticks()]        
        _ = [tick.label.set_rotation(-75) for tick in ax.xaxis.get_major_ticks()]        
    else:
        ax.set_axis_off()

    ax_list = [2, 3, 5, 6]
    
    print('****************************** Subgraph {} ******************************'.format(fac_ix+1))    
    print(f_sys_subgraph['system_string'])

    # Handle Distribution Plotting
    fstat = fac_stat[fac_ix]
    for subp_ii, gstat in enumerate(fstat):
        ax = plt.subplot(3, 3, ax_list[subp_ii])
        ax.plot(gstat['distrib'][0], 
                gstat['stats'][0]['slope']*gstat['distrib'][0]+gstat['stats'][0]['interc'],
                color='k', alpha=0.75)
        ax.scatter(gstat['distrib'][0], gstat['distrib'][1],
                   color=[0.3, 0.3, 0.3])

        ax.set_xlabel('Blocks with Active Expression')
        ax.set_ylabel('Interference Accuracy')
        ax.set_title(gstat['title'])
        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')

        for kstat in gstat['stats']:
            print('    %12s: rho: %3.5f pval: %3.5f sig: %s' % 
              (gstat['title'],
               kstat['rho'],
               kstat['pval'],
               kstat['sig']))
        print('')
            
    plt.show()
    print('\n\n\n')    

## Predicting Cost Accuracy (Hi-Lo)

In [ ]:
%matplotlib inline

grp_order = ['Stroop', 'Navon']
tsk_order = ['pos', 'neg']
beh_grp = {'Stroop': [stroop_move, stroop_cost_acc],
           'Navon': [navon_move, navon_cost_acc]}

# Run Statistics on Key and Group Level from Above
# Accumulate in fac_stat
fac_stat = []
for fac_ii in xrange(n_subgraph):
    grp_stat = []
                 
    for grp in grp_order:
        for tsk in tsk_order:
            key_hi_ix = np.flatnonzero(key_type == 'adj_hi_{}_{}'.format(tsk, grp))
            key_lo_ix = np.flatnonzero(key_type == 'adj_lo_{}_{}'.format(tsk, grp))
            
            subj_coef_hi = fac_coef_subj[fac_ii, :, key_hi_ix, :][0, ...]
            subj_coef_lo = fac_coef_subj[fac_ii, :, key_lo_ix, :][0, ...]
            mean_subj_coef = np.mean(subj_coef_hi, axis=1) - np.mean(subj_coef_lo, axis=1)
            
            m, b, _, _, _ = stats.linregress(beh_grp[grp][0], mean_subj_coef)
            real_mean_subj_coef_nomove = mean_subj_coef - beh_grp[grp][0]*m+b
            
            m, b, _, _, _ = stats.linregress(beh_grp[grp][0], beh_grp[grp][1])
            beh_score_nomove = beh_grp[grp][1] - beh_grp[grp][0]*m+b            
            
            real_m, real_b, real_rho, real_pval, _ = stats.linregress(real_mean_subj_coef_nomove, beh_score_nomove)
            
            null_rho_dist = []
            for i in xrange(10000):
                null_rho, _ = stats.pearsonr(np.random.permutation(real_mean_subj_coef_nomove),
                                             beh_score_nomove)
                null_rho_dist.append(null_rho)
            null_rho_dist = np.array(null_rho_dist)
            if real_rho > 0:
                real_pval = np.mean(null_rho_dist > real_rho)
            else:
                real_pval = np.mean(null_rho_dist < real_rho)
                
            subplot_dict = {'title': '{}: {}'.format(grp, tsk),
                            'xlabel': tsk,
                            'distrib': (real_mean_subj_coef_nomove, beh_score_nomove),
                            'stats': []}
            subplot_dict['stats'].append({'rho': real_rho,
                                          'pval': real_pval,
                                          'slope': real_m,
                                          "interc": real_b,
                                          'sig': ''})       
            
            grp_stat.append(subplot_dict)
    fac_stat.append(grp_stat)

# Benjamini-Hochberg Multiple Comparisons
all_pval = np.sort([kstat['pval']
                    for fstat in fac_stat
                    for gstat in fstat
                    for kstat in gstat['stats']])
alpha = 0.05
n_comp = len(all_pval)
for ix, i in enumerate(range(1, n_comp+1, 1)):
    adj_alpha = alpha/(n_comp+1-i)
    if all_pval[ix] > adj_alpha:
        benj_hoch_ix = ix
        break
print('Adjusted Alpha: %0.10f' % adj_alpha)

# Re-adjust significant indicator and re-order subgraphs
min_fac_pval = []
sel_fac_between_sig = []
for fac_ii, fstat in enumerate(fac_stat):
    pval_list = []
    for gstat in fstat:
        for kstat in gstat['stats']:
            if kstat['pval'] in all_pval[:benj_hoch_ix]:
                kstat['sig'] = '*'
            pval_list.append(kstat['pval'])
    min_fac_pval.append(np.min(pval_list))
sel_fac_ord = np.argsort(min_fac_pval)

# Plot each result
for fac_ix in sel_fac_ord[:10]:
    
    # Handle Subgraph Plotting
    f_sys_subgraph = sys_subgraph[fac_ix]
    #assert f_sys_subgraph['Subgraph_ID'] == (fac_ix+1)
    
    plt.figure(figsize=(12,12))
    ax = plt.subplot(3, 3, 1)
    ax.matshow(f_sys_subgraph['filt_system_subg'], cmap='rainbow')
    if len(df_subg['system_names']) < 20:
        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')

        ax.set_yticks(np.arange(len(df_subg['system_names'])))
        ax.set_xticks(np.arange(len(df_subg['system_names'])))
        ax.set_yticklabels(df_subg['system_names'])
        ax.set_xticklabels(df_subg['system_names'])
        _ = [tick.label.set_fontsize(12) for tick in ax.yaxis.get_major_ticks()]
        _ = [tick.label.set_fontsize(12) for tick in ax.xaxis.get_major_ticks()]        
        _ = [tick.label.set_rotation(-75) for tick in ax.xaxis.get_major_ticks()]        
    else:
        ax.set_axis_off()

    ax_list = [2, 3, 5, 6]
    
    print('****************************** Subgraph {} ******************************'.format(fac_ix+1))    
    print(f_sys_subgraph['system_string'])

    # Handle Distribution Plotting
    fstat = fac_stat[fac_ix]
    for subp_ii, gstat in enumerate(fstat):
        ax = plt.subplot(3, 3, ax_list[subp_ii])
        ax.plot(gstat['distrib'][0], 
                gstat['stats'][0]['slope']*gstat['distrib'][0]+gstat['stats'][0]['interc'],
                color='k', alpha=0.75)
        ax.scatter(gstat['distrib'][0], gstat['distrib'][1],
                   color=[0.3, 0.3, 0.3])

        ax.set_xlabel('Blocks with Active Expression (Hi-Lo)')
        ax.set_ylabel('Cost Accuracy')
        ax.set_title(gstat['title'])
        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')

        for kstat in gstat['stats']:
            print('    %12s: rho: %3.5f pval: %3.5f sig: %s' % 
              (gstat['title'],
               kstat['rho'],
               kstat['pval'],
               kstat['sig']))
        print('')
            
    plt.show()
    print('\n\n\n')    